In [317]:
import random
from math import pi, sqrt, cos, sin,exp
from matplotlib import pyplot as plt


def gaussian(mu,sigma,x):
    return exp(-((mu-x)**2)/(sigma**2)/2.0)/sqrt(2.0*pi*(sigma**2))

class Map():
    def __init__(self, height, width, landmarks, weight=1.0):
#         self.width = width
#         self.height = height
        self.landmarks = landmarks
        
ALL_MULTS =[]
class Particle():
    distance_sigma = 0.5
    def __init__(self, orientation, x, y, weight=1.0):
        self.x = x
        self.y = y
        self.orientation = orientation
        self.weight = weight
        
    def move(self, u, noise, world_map):
        distance, turn = u
        distance += random.gauss(0,noise[0])
        turn += random.gauss(0,noise[1] )
        self.orientation = (self.orientation + turn + 2*pi) % (2*pi)
        dx = distance * cos(self.orientation)
        dy = distance * sin(self.orientation)
        self.x += dx
#         self.x %= world_map.width
        
        self.y += dy
#         self.y %= world_map.height
        
    def update_weight(self, observations, landmarks):
        self.weight = 1.0
        for lm_id, lm in landmarks.items():
            obs = observations[lm_id]
            true_dx, true_dy = obs
            true_dist = sqrt(true_dx ** 2 + true_dy ** 2)
            
            lm_x, lm_y = lm
            expected_dx = lm_x - self.x
            expected_dy = lm_y - self.y
            
            expected_dist = sqrt(expected_dx ** 2 + expected_dy ** 2)
            diff = true_dist - expected_dist
            multiplier = gaussian(0, self.distance_sigma, diff)
            ALL_MULTS.append(multiplier)
            self.weight *= multiplier
        self.weight += 0.000001
    
    def normalize_weight(self, factor):
        self.weight /= factor
        
    def __repr__(self):
        s = "\nx:      {}\n".format(self.x)
        s +="y:      {}\n".format(self.y)
        s +="theta:  {}\n".format(self.orientation)
        s +="weight: {}\n".format(self.weight)
        return s
        

class Robot(Particle):
    def sense(self, world_map):
        observed = {}
        landmarks = world_map.landmarks
        for lm_id, lm in landmarks.items():
            dx = lm[0] - self.x
            dy = lm[1] - self.y
            observed[lm_id] = (dx, dy)
        return observed

class ParticleFilter():
    N = 1000
    sigma_theta = 2*pi / 180.0
    sigma_distance = 1.2
    
    motion_noise = (2*pi / 180.0, 0.1)
    
    def __init__(self, world_map):
        self.map = world_map
#         self.particles = self._initialize_particles()
        pass
    
    def initialize(self, theta, x, y):
        particles = []
        for i in range(self.N):
            orientation = random.gauss(theta, self.sigma_theta) % (2*pi)
            X = random.gauss(x, self.sigma_distance) #% self.map.width
            Y = random.gauss(y, self.sigma_distance) #% self.map.height
            particles.append(Particle(orientation, X, Y))
        self.particles = particles
    
    def move(self, u):
        for p in self.particles:
            theta_noise = random.gauss(0, self.motion_noise[0])
            distance_noise = random.gauss(0, self.motion_noise[1])
            noise = (distance_noise, theta_noise)
            p.move(u,noise,self.map)
        pass
    
    def sense(self, observations):
        self._update_weights(observations)
        self._resample()
        self._normalize()
        pass
    
    def _update_weights(self, observations):
        landmarks = self.map.landmarks
        total_weight = 0.0
        for particle in self.particles:
            particle.update_weight(observations, landmarks)
            total_weight += particle.weight
        print "totalweight before: {}".format(total_weight)
        normalization_factor = total_weight / self.N
        print "normalization factor: {}".format(normalization_factor)
        for particle in self.particles:
            particle.normalize_weight(normalization_factor)
        print "totalweight before: {}".format(sum([p.weight for p in self.particles]))
            
    def _resample(self):
        new_particles = []
        for i in range(self.N):
            new_particle = weighted_choice(self.particles)
            new_particles.append(new_particle)
        self.particles = new_particles     
        
    def _normalize(self):
        for p in self.particles:
            p.weight += 0.000001
        total = sum(p.weight for p in self.particles)
        factor = total / self.N
        for p in self.particles:
            p.normalize_weight(factor)
    
    def _initialize_particles(self):
        particles = []
        for i in range(self.N):
            x = random.randrange(self.map.width)
            y = random.randrange(self.map.height)
            theta = random.uniform(0, 2*pi )
            particles.append(Particle(theta, x, y))
        return particles
    
    def best_guess(self):
        best_particle = max(self.particles, key=lambda p: p.weight)
        return best_particle
    
    def show(self, robot=None):
        X = [p.x for p in self.particles]
        Y = [p.y for p in self.particles]
        plt.scatter(X,Y)
        if robot:
            plt.scatter([robot.x], [robot.y],color='red')
#         plt.xlim = (0,self.map.width)
#         plt.ylim = (0,self.map.height)
        plt.axis([0,10, 0, 10])
        plt.show()

def weighted_choice(particles):
    total = sum(p.weight for p in particles)
    r = random.uniform(0, total)
    upto = 0.0
    for p in particles:
        w = p.weight
        if upto + w >= r:
            return p
        upto += w
    assert False, "Shouldn't get here"

In [326]:
LANDMARKS = {
    0: (5,5),
    1: (3,3),
    2: (7,3),
    3: (3,7),
    4: (7,7),
    5: (10,10),
    6: (0,0),
    7: (0,10),
    8: (10,0)
}
MAP = Map(10,10,LANDMARKS)

def test_particle_filter(world_map):
    robot = Robot(0,2,2)
    pf = ParticleFilter(world_map)
    pf.initialize(robot.orientation, robot.x, robot.y)
    for i in range(50):
        update_simulation(world_map, robot, pf)
        
def update_simulation(world_map, robot, pf):
    observations = robot.sense(world_map)
    pf.sense(observations)
    
    motion = (0.3 , pi*2 / 16.0)
    noise = (0.0, 0.0)
    robot.move(motion, noise, world_map)
    pf.move(motion)
    
    best_particle = pf.best_guess()
    print "robot: {}, {}".format(robot.x, robot.y)
    print "part:  {}, {}".format(best_particle.x, best_particle.y)
    print

In [327]:
start = (0,3,3)
robot = Robot(*start)
pf = ParticleFilter(MAP)
pf.initialize(*start)
pf.show(robot)

In [329]:
motion = (pi*2 / 20, 0.5)
noise = (0,0)
robot.move(motion,noise, MAP)
pf.move(motion)
pf.show(robot)
observations = robot.sense(MAP)
pf.sense(observations)
pf.show(robot)

totalweight before: 4.33156409077
normalization factor: 0.00433156409077
totalweight before: 4.01082702281e+67


In [302]:
sum(p.weight for p in pf.particles)

7.101690897299372e+37

In [301]:
pf._normalize()

In [303]:
weighted_choice(pf.particles)


x:      3.32158841836
y:      3.05383756031
theta:  0.605098639153
weight: 1.1803391992e+36

In [304]:
min(pf.particles, key=lambda p: p.weight)


x:      3.81287284604
y:      3.77276981639
theta:  0.50242161025
weight: 1.4456207149

In [322]:
plt.hist(ALL_MULTS, 200)
plt.show()